# Sample Summaries  

## Tagged Colony Summaries 

In [1]:
library(dplyr)
library(tidyverse)
library(ggplot2)
library(tidyr)
library(purrr)
library(stringr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.2     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.2.1
✔ purrr     1.0.4     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
colony <- read_csv("/Users/brookesienkiewicz/Documents/sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv", show_col_types = FALSE)

New names:
• `` -> `...1`


In [3]:
# change orbi rapid colony to ofav for now 
unique(colony$Species)
colony[colony$Species=='ORBI','Species']<- "OFAV"
unique(colony$Species)

[1] "SSID" "PAST" "PSTR" "MCAV" "OANN" "DLAB" "MMEA" "OFAV" "CNAT"

[1] "SSID" "PAST" "PSTR" "MCAV" "OANN" "DLAB" "MMEA" "OFAV" "CNAT"

### All tagged colonies

#### By transect

In [4]:
# summary of tagged colonies 
tagged_summarytable<-colony %>%
  group_by(Transect, Species) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0)

    # Add row totals
    tagged_summarytable <- tagged_summarytable %>%
      mutate(Total = rowSums(select(., -Transect)))
    
    # Add column totals
    totals_row <- tagged_summarytable %>%
      select(-Transect) %>%
      summarise(across(everything(), sum)) %>%
      mutate(Transect = "Total") %>%
      select(Transect, everything())
    
    # Combine
    tagged_summarytable <- bind_rows(tagged_summarytable, totals_row)
    tagged_summarytable

Transect,MCAV,OANN,OFAV,PAST,PSTR,SSID,DLAB,MMEA,CNAT,Total
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
BB,6,7,6,6,6,6,0,0,0,37
CBC30N,9,3,0,5,7,8,2,2,0,36
CURLEW,5,1,4,0,5,0,2,0,0,17
HANGMAN,5,5,5,5,6,5,0,0,0,31
LAGOON,10,2,3,9,10,10,2,5,0,51
SR30N,7,2,4,6,9,11,2,6,1,48
Total,42,20,22,31,43,40,8,13,1,220


In [5]:
# add reef location grouping 
colony<-colony %>%
    mutate(reef_location = case_when(
           Transect %in% c('BB','HANGMAN') ~ "Inshore",
           Transect %in% c('CBC30N','CURLEW','LAGOON','SR30N') ~ "Offshore"
        ))

### Bleaching (2023-)
- includes only colonies alive in 9/2023 onward 

In [6]:
# remove colonies that died before bleaching event 
unique(colony$Date_DocumentedMortality)
class(colony$Date_DocumentedMortality)

[1] "Diseased"               "Healthy"                "12/2/22"               
 [4] "9/25/23"                "12/1/24"                "12/16/23"              
 [7] "4/1/24"                 "4/26/24"                "8/1/24"                
[10] "5/21/22"                "9/26/23"                "12/1/22"               
[13] "5/22/22"                "12/4/22"                "1/11/24"               
[16] "6/21/24"                "5/20/22"                "1/1/24"                
[19] "5/24/23"                "12/3/22"                NA                      
[22] "Could not find 9/26/23"

[1] "character"

In [7]:
# viewing colonies that died in/around 9/2023
colony %>% 
    filter(Date_DocumentedMortality=='5/24/23') %>%
    select(contains("Condition"))
colony %>% 
    filter(str_detect(Date_DocumentedMortality, "^9/.*/23$")) %>%
    select(contains("Condition"))

# do i include dead colonies in 9/23? no? 

062019_Condition,052022_Condition,122022_Condition,092023_Condition,112023_Condition,122023_Condition,012024_Condition,022024_Condition,042024_Condition,062024_Condition,082024_Condition,122024_Condition,062025_Condition
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Healthy,Diseased,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,Not_visited


062019_Condition,052022_Condition,122022_Condition,092023_Condition,112023_Condition,122023_Condition,012024_Condition,022024_Condition,042024_Condition,062024_Condition,082024_Condition,122024_Condition,062025_Condition
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Healthy,Healthy,Diseased,Dead,Not_Visited,Not_Visited,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Dead,Dead
Healthy,Diseased,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Dead,Not_Visited,Not_Visited,Not_visited,Dead
Healthy,Diseased,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,Dead
NA,NA,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,NA
Healthy,Healthy,Healthy,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,NA
NA,NA,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,Not_visited
NA,NA,Healthy,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,Not_visited
NA,NA,Healthy,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,NA
NA,NA,Diseased,Dead,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_Visited,Not_visited,Not_Visited,Not_visited,NA


In [8]:
clb_colony<-colony %>%
    # remove colonies that died in 2022 and before 9/23
  filter(
    !str_detect(Date_DocumentedMortality, "/22") &
    !str_detect(Date_DocumentedMortality, "9/.*/23") &
    !str_detect(Date_DocumentedMortality, "5/.*/23")
  )
     
unique(clb_colony$Date_DocumentedMortality)

[1] "Diseased" "Healthy"  "12/1/24"  "12/16/23" "4/1/24"   "4/26/24" 
 [7] "8/1/24"   "1/11/24"  "6/21/24"  "1/1/24"

#### All - species, bleaching status

In [9]:
# Add bleach status column
clb_colony <- clb_colony %>%
  mutate(
    clb_y_n = if_any(everything(), ~ str_detect(., "CLB|CLP")),
    Status = case_when(
    clb_y_n ~ "Bleached",
    is.na(clb_y_n) ~ "Not Bleached", 
    TRUE ~ "Not Bleached"
    ))

In [10]:
# Count by Location, Species, and Status
clb_specie <- clb_colony %>%
  group_by(Species, Status) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0) %>%
  mutate(Total = rowSums(across(-c(Status))))

# Add grand total row
grand_total <- clb_specie %>%
  select(-c(Status)) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Status = "Total") %>%
  select(Status, everything())

# Combine
clb_summary <- bind_rows(clb_specie, grand_total)
clb_summary

Status,CNAT,DLAB,MCAV,OANN,OFAV,PAST,PSTR,SSID,Total
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Bleached,1,5,18,18,14,21,19,15,111
Not Bleached,0,3,7,2,7,7,2,20,48
Total,1,8,25,20,21,28,21,35,159


#### All - by reef location, specie, bleaching status 
- reef location = inshore (BB, Hangman) or offshore (T1-T4)

In [11]:
# Count by Location, Species, and Status
clb_locationsum <- clb_colony %>%
  group_by(reef_location, Species, Status) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0) %>%
  mutate(Total = rowSums(across(-c(reef_location, Status))))

# Add row totals per Status
status_totals <- clb_locationsum %>%
  select(-reef_location) %>%
  group_by(Status) %>%
  summarise(across(everything(), sum)) %>%
  mutate(reef_location = "Total") %>%
  select(reef_location, everything())

# Add grand total row
grand_total <- clb_locationsum %>%
  select(-c(reef_location, Status)) %>%
  summarise(across(everything(), sum)) %>%
  mutate(reef_location = "Total", Status = "Total") %>%
  select(reef_location, Status, everything())

# Combine
clb_location_summary <- bind_rows(clb_locationsum, status_totals, grand_total) %>%
  arrange(reef_location, Status)

clb_location_summary

reef_location,Status,MCAV,OANN,OFAV,PAST,PSTR,SSID,CNAT,DLAB,Total
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Inshore,Bleached,7,11,9,7,10,1,0,0,45
Inshore,Not Bleached,4,1,2,4,2,10,0,0,23
Offshore,Bleached,11,7,5,14,9,14,1,5,66
Offshore,Not Bleached,3,1,5,3,0,10,0,3,25
Total,Bleached,18,18,14,21,19,15,1,5,111
Total,Not Bleached,7,2,7,7,2,20,0,3,48
Total,Total,25,20,21,28,21,35,1,8,159


#### All - by transect, specie, bleaching status 

In [12]:
# Count by Transect, Species, and Status
summary_tbl <- clb_colony %>%
  group_by(Transect, Species, Status) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0) %>%
  mutate(Total = rowSums(across(-c(Transect, Status))))

# Add row totals per Status
status_totals <- summary_tbl %>%
  select(-Transect) %>%
  group_by(Status) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Transect = "Total") %>%
  select(Transect, everything())

# Add grand total row
grand_total <- summary_tbl %>%
  select(-c(Transect, Status)) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Transect = "Total", Status = "Total") %>%
  select(Transect, Status, everything())

# Combine
clb_summary_transect <- bind_rows(summary_tbl, status_totals, grand_total) %>%
  arrange(Transect, Status)

clb_summary_transect

Transect,Status,MCAV,OANN,OFAV,PAST,PSTR,SSID,DLAB,CNAT,Total
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
BB,Bleached,4,7,6,6,4,0,0,0,27
BB,Not Bleached,2,0,0,0,2,6,0,0,10
CBC30N,Bleached,2,3,0,5,1,2,1,0,14
CBC30N,Not Bleached,0,0,0,0,0,5,1,0,6
CURLEW,Bleached,2,0,2,0,2,0,1,0,7
CURLEW,Not Bleached,1,1,2,0,0,0,1,0,5
HANGMAN,Bleached,3,4,3,1,6,1,0,0,18
HANGMAN,Not Bleached,2,1,2,4,0,4,0,0,13
LAGOON,Bleached,3,2,0,7,4,3,2,0,21


#### RAPID

##### RAPID - specie, bleaching status

In [13]:
immune_summarytable<-clb_colony %>%
  filter(`immune_y/n`=='y') %>%
  group_by(Species, Status) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0) %>%
  mutate(Total = rowSums(across(-c(Status))))
  
# Add grand total row
grand_total <- immune_summarytable %>%
  select(-c(Status)) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Status = "Total") %>%
  select(Status, everything())

# Combine
rapid_summary <- bind_rows(immune_summarytable, grand_total)
rapid_summary

Status,MCAV,OANN,OFAV,PAST,PSTR,SSID,Total
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Bleached,9,1,2,8,5,6,31
Not Bleached,1,0,3,1,0,0,5
Total,10,1,5,9,5,6,36


##### RAPID - specie, transect 
- no RAPID colonies at inshore sites 

In [14]:
# summary of immune tagged colonies 
immune_summarytable<-clb_colony %>%
  filter(`immune_y/n`=='y') %>%
  group_by(Transect, Species) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0)

    # Add row totals
    immune_summarytable <- immune_summarytable %>%
      mutate(Total = rowSums(select(., -Transect)))
    
    # Add column totals
    totals_row <- immune_summarytable %>%
      select(-Transect) %>%
      summarise(across(everything(), sum)) %>%
      mutate(Transect = "Total") %>%
      select(Transect, everything())
    
    # Combine
    immune_summarytable <- bind_rows(immune_summarytable, totals_row)
    immune_summarytable

Transect,MCAV,OANN,PAST,PSTR,SSID,OFAV,Total
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
CBC30N,1,1,3,1,1,0,7
CURLEW,3,0,0,2,0,1,6
LAGOON,3,0,3,2,2,2,12
SR30N,3,0,3,0,3,2,11
Total,10,1,9,5,6,5,36


##### RAPID grouped by transect, bleaching status

In [15]:
# repeat for immune 
immune <-clb_colony %>%
  filter(`immune_y/n`=='y')
      
# Count by Transect, Species, and Status
summary_tbl <- immune %>%
  group_by(Transect, Species, Status) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0) %>%
  mutate(Total = rowSums(across(-c(Transect, Status))))

# Add row totals per Status
status_totals <- summary_tbl %>%
  select(-Transect) %>%
  group_by(Status) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Transect = "Total") %>%
  select(Transect, everything())

# Add grand total row
grand_total <- summary_tbl %>%
  select(-c(Transect, Status)) %>%
  summarise(across(everything(), sum)) %>%
  mutate(Transect = "Total", Status = "Total") %>%
  select(Transect, Status, everything())

# Combine
clb_immunesummary <- bind_rows(summary_tbl, status_totals, grand_total) %>%
  arrange(Transect, Status)

clb_immunesummary

Transect,Status,MCAV,OANN,PAST,PSTR,SSID,OFAV,Total
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
CBC30N,Bleached,1,1,3,1,1,0,7
CURLEW,Bleached,2,0,0,2,0,1,5
CURLEW,Not Bleached,1,0,0,0,0,0,1
LAGOON,Bleached,3,0,3,2,2,0,10
LAGOON,Not Bleached,0,0,0,0,0,2,2
SR30N,Bleached,3,0,2,0,3,1,9
SR30N,Not Bleached,0,0,1,0,0,1,2
Total,Bleached,9,1,8,5,6,2,31
Total,Not Bleached,1,0,1,0,0,3,5


In [18]:
# export 

## summary tables for aim 1 (bleaching demographics)
### All - species, bleaching status
write.csv(clb_summary, 'colony_tables/clb_colonysummary.csv',row.names=FALSE)
### All - by reef location, specie, bleaching status 
write.csv(clb_location_summary, 'colony_tables/clb_colonysummary_location.csv',row.names=FALSE)

#### RAPID - specie, bleaching status
write.csv(rapid_summary, 'colony_tables/clb_RAPIDSummary.csv',row.names=FALSE)

In [ ]:
## Bleaching Sample Summaries is in BleachedSamples.ipynb 

### SCTLD 

#### tagged colony summary that were sequenced in brooke's 03/2025 set

In [48]:
# input sctld_samples spreadsheet 
sctld_samples <- read.csv("/Users/brookesienkiewicz/Documents/sctld/SCTLD_samples/Sample_Data/CBC_samples.csv")

# input list of sequenced samples (3/2025 set)
samplelist<- read.table('../inputs/genohublist_sctld2024.txt')
colnames(sctld_samples)

[1] "Month_year"                   "Country"                     
 [3] "Location"                     "CollectionDate"              
 [5] "Transect"                     "TransectNum"                 
 [7] "OldTagNum"                    "NewTagNum"                   
 [9] "Species"                      "Time_sampled"                
[11] "Time_processed"               "Sample_type"                 
[13] "SampleNum"                    "Health_status"               
[15] "Sampling_notes"               "Tubelabel_species"           
[17] "Sample_physical_location"     "Extraction_physical_location"
[19] "Date_sequenced"               "Notes"

In [49]:
#filter sample data to match sequence list 
sctld_samples<-sctld_samples %>%
    filter(Sample_type == 'Core_EtOH') %>%
    filter(Tubelabel_species %in% samplelist$V1)
nrow(sctld_samples)

[1] 220

In [50]:
# what is in samplelist that is not in sctld_samples (filtered)
samplelist[!samplelist$V1 %in% sctld_samples$Tubelabel_species,]

nrow(samplelist)
# extra 2 in seq list are negative controls 

[1] "7_3_Neg"  "7_11_Neg"

[1] 222

In [51]:
# make colony tag ID for sample data 
transect_id <- paste0('T',sctld_samples$TransectNum)
sctld_samples$colony_id <- paste(transect_id, sctld_samples$NewTagNum, sctld_samples$Species,
                                 sep = "_")
nrow(sctld_samples)
length(unique(sctld_samples$colony_id))
# 220 samples from 114 total colonies?

[1] 220

[1] 114

In [52]:
# make colony tag ID for colony data 
transect_id <- paste0('T',colony$TransectNum)
colony$colony_id <- paste(transect_id, colony$NewTagNum, colony$Species,
                                 sep = "_")

In [53]:
# filter colony data for colonies that had samples that were sequenced  
sctld_colony <- colony %>%
    filter(colony_id %in% sctld_samples$colony_id)
length(unique(sctld_colony$colony_id))

[1] 113

In [54]:
# missing 5 colonies?
    ## (after updates listed below, only 1 different colony - an accidental sample that was sequenced)
    ## need to remove AS from metagenomics analysis 

# which colony IDs in sample data are missing from colony data? 
sctld_samples %>%
    filter(!colony_id %in% sctld_colony$colony_id)

# mmea colony tag nums for some reason got changed in colonydata but not cbc_samples 
# don't know about T3-41-ofav, t1-25-oann, and t2-56-mcav ??
# all tubelabels are in genohub sample list - so everything has been sequenced 

# can leave the AS excluded 
# updated t3-41 ofav to tag #66. see SCTLD_samples CBC_samples commit history for more details 8/9/2025
# t1-25 oann - changed species to oann on both datasheets - see ^ & colonydata commit history 8/9/2025
# t2-56 mcav - tag num was listed as 0 in colony data. - see ^ & colonydata commit history 8/9/2025
# reverted mmea tag nums to match old tag nums in colony data. - see ^ & colonydata commit history 8/9/2025

Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,Species,Time_sampled,⋯,Sample_type,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes,colony_id
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
122022,BEL,CBC,12/4/22,SR30N,2,,AS,OANN,,⋯,Core_EtOH,107,Diseased_Tissue,,122022_BEL_CBC_T2_107_OANN,Depleted_UML_NARWHAL_R1_B4,UML_NARWHAL_R2_B3,,Only sand left in tissue sample,T2_AS_OANN


In [55]:
# add resistant, susceptible, sctld_mortality, or sctld_recovery health statuses for each colony
# specify columns to check 
cols<- c('062019_Condition','052022_Condition','122022_Condition','092023_Condition')
disease_cols<- c('052022_Condition','122022_Condition')
recovery_cols<-c('122022_Condition','092023_Condition')

# add health statuses 
sctld_colony <- sctld_colony %>%
# make smaller df
    select('Date_InitialTag','Transect','Species','colony_id',cols) %>%
    mutate(
        health_status = case_when(
            # if all cols healthy ~ resis, 
            # if diseased then dead ~ SCTLD_Mortality, 
            # if diseased then healthy ~ sctld_recovery, 
            # if just disease and/or bl ~ sus
            # include statuses in 9/2023 to incl any diseased in 5 and 122022 that died later on 
            if_any(any_of(cols), ~ str_detect(., "Dead")) ~ "Sus_mortality",
            if_any(any_of(disease_cols), ~ str_detect(., "Diseased")) &  if_any(any_of(recovery_cols), ~ str_detect(., "Healthy")) ~ "Sus_recovery",
            if_any(any_of(cols), ~ str_detect(., "Diseased")) ~ "Susceptible",
            if_any(any_of(cols), ~ str_detect(., "Healthy")) ~ "Resistant",
            TRUE ~ "Resistant")
        )

In [56]:
sctld_colony
unique(sctld_colony$health_status)
# view health statuses over time

Date_InitialTag,Transect,Species,colony_id,062019_Condition,052022_Condition,122022_Condition,092023_Condition,health_status
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
6/21/19,CBC30N,PAST,T1_2_PAST,Healthy,Healthy,Healthy,"CLP,CLB",Resistant
10/15/19,CBC30N,PSTR,T1_4_PSTR,Healthy,Diseased,Dead,Not_Visited,Sus_mortality
10/15/19,CBC30N,PSTR,T1_6_PSTR,Healthy,Healthy,Diseased,Dead,Sus_mortality
6/24/19,CBC30N,MCAV,T1_7_MCAV,Healthy,Healthy,Healthy,CLP,Resistant
6/24/19,CBC30N,MCAV,T1_8_MCAV,Healthy,Diseased,Diseased,Dead,Sus_mortality
10/15/19,CBC30N,PSTR,T1_12_PSTR,Healthy,Healthy,Healthy,Healthy,Resistant
6/24/19,CBC30N,PAST,T1_13_PAST,Healthy,Healthy,Healthy,Healthy,Resistant
6/24/19,CBC30N,MCAV,T1_14_MCAV,Healthy,Diseased,Dead,Not_Visited,Sus_mortality
6/26/19,CBC30N,MCAV,T1_15_MCAV,Healthy,Diseased,Dead,Not_Visited,Sus_mortality


[1] "Resistant"     "Sus_mortality" "Sus_recovery"  "Susceptible"

##### bks set - species, status:resis or sus 

In [57]:
# summary by health statuses 
summary_sctld <- sctld_colony %>%
  group_by(Species, health_status) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0)%>%
# add totals per health status 
  mutate(Total = rowSums(across(-c(health_status))))

# Add row totals per Status
status_totals <- summary_sctld %>%
  select(-health_status) %>%
  summarise(across(everything(), sum)) %>%
  mutate(health_status = "Total") %>%
  select(everything())

In [58]:
# Combine
sctld_summarytable <- bind_rows(summary_sctld, status_totals) %>%
    arrange(factor(health_status, levels = c('Resistant','Susceptible','Sus_recovery','Sus_mortality','Total')))
sctld_summarytable

health_status,MCAV,MMEA,OANN,OFAV,PAST,PSTR,Total
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Resistant,13,0,5,8,16,8,50
Susceptible,0,0,2,3,0,1,6
Sus_recovery,1,0,0,0,4,0,5
Sus_mortality,17,13,0,0,0,22,52
Total,31,13,7,11,20,31,113


In [59]:
# combine disease and dead by disease to compare just resistant and susceptible
# ..should i leave sctld_recovery as its own category or include it in resistant?
disease_totals <- summary_sctld %>%
    filter(health_status %in% c('Susceptible','Sus_mortality')) %>%
    select(-health_status) %>%
    summarise(across(everything(), sum)) %>%
    mutate(health_status = "Susceptible") %>%
    select(everything())
# drop cols
healthy_sctld<-sctld_summarytable %>%
    filter(!health_status %in% c('Susceptible','Sus_mortality'))
#combine 
sctld_summarytable2 <- bind_rows(healthy_sctld,disease_totals) %>%
    arrange(factor(health_status, levels = c('Resistant','Susceptible','Sus_recovery','Total')))
sctld_summarytable2

health_status,MCAV,MMEA,OANN,OFAV,PAST,PSTR,Total
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Resistant,13,0,5,8,16,8,50
Susceptible,17,13,2,3,0,23,58
Sus_recovery,1,0,0,0,4,0,5
Total,31,13,7,11,20,31,113


In [60]:
# export 
write.csv(sctld_summarytable2, 'colony_tables/bks_sctld_colonysummary.csv', row.names = FALSE)

In [61]:
colnames(sctld_colony)

[1] "Date_InitialTag"  "Transect"         "Species"          "colony_id"       
[5] "062019_Condition" "052022_Condition" "122022_Condition" "092023_Condition"
[9] "health_status"

In [62]:
# convert monthyear into date format 

# add leading 0 
sctld_samples$Month_year<-sprintf("%06d", sctld_samples$Month_year)
# add day 
sctld_samples$Month_year<-paste0(sctld_samples$Month_year, "01")
# convert to date format 
sctld_samples$Month_year <- as.Date(sctld_samples$Month_year, format = "%m%Y%d")
# convert to mm/yyyy format 
sctld_samples$Month_year<-format(sctld_samples$Month_year, "%m/%Y")



In [63]:
unique(sctld_samples$Month_year)

[1] "12/2022" "05/2022" "10/2019" "06/2019"

In [64]:
# combine 062019 and 102019
sctld_samples$Month_year[sctld_samples$Month_year %in% c("06/2019", "10/2019")] <- "2019"
unique(sctld_samples$Month_year)

[1] "12/2022" "05/2022" "2019"

In [65]:
# tracking colonies over time? 
# how many do we have in 2019, 5/2022, and 12/2022

# need to use sctld_samples for this instead?
sctld_samples<- sctld_samples %>%
    # filter out the AS 
    filter(!NewTagNum == 'AS')

# add health_status to sctld_samples by matching colony id to colony data 
sctld_samples$health_status <- sctld_colony$health_status[match(sctld_samples$colony_id, sctld_colony$colony_id)]

# group by specie, health status, and year?


In [66]:
sctld_samples$Month_year<-factor(sctld_samples$Month_year, levels = c("2019","05/2022","12/2022")) 

test <- sctld_samples %>%
  group_by(Species, health_status, Month_year) %>%
  summarise(n = n(), .groups = "drop") %>%
# make date the columns 
# sort first 
  arrange(Month_year) %>%
  pivot_wider(names_from = Month_year, values_from = n, values_fill = 0)
# add totals per health status 
  # mutate(Total = rowSums(across(-c(health_status))))
test

# ya idk why these numbers are so wack
# and this isn't unique colonies over time it's just how many colonies at each time point 

Species,health_status,2019,05/2022,12/2022
<chr>,<chr>,<int>,<int>,<int>
MCAV,Resistant,9,9,13
MCAV,Sus_mortality,14,13,8
MCAV,Sus_recovery,1,0,1
MMEA,Sus_mortality,13,0,0
PAST,Resistant,16,9,12
PAST,Sus_recovery,4,6,6
PSTR,Resistant,3,3,8
PSTR,Sus_mortality,17,9,11
PSTR,Susceptible,1,1,1


##### all tagged summary - species, status:resis or sus 
- all colonies tagged 2019 & 2022 fate tracking 

In [67]:
# filter for colonies tagged pre 2024  
allsctld_colony<-colony %>%
    filter(!str_ends(Date_InitialTag, "/24"))

In [68]:
allsctld_colony <- allsctld_colony %>%
  mutate(
    health_status = case_when(
            if_any(any_of(cols), ~ str_detect(., "Dead")) ~ "Sus_mortality",
            if_any(any_of(disease_cols), ~ str_detect(., "Diseased")) &  if_any(any_of(recovery_cols), ~ str_detect(., "Healthy")) ~ "Sus_recovery",
            if_any(any_of(cols), ~ str_detect(., "Diseased")) ~ "Susceptible",
            if_any(any_of(cols), ~ str_detect(., "Healthy")) ~ "Resistant",
            TRUE ~ "Resistant")
        )

# summarize 
summary_allsctld <- allsctld_colony %>%
  group_by(Species, health_status) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0)%>%
# add totals per health status 
  mutate(Total = rowSums(across(-c(health_status))))

# Add row totals per Status
allstatus_totals <- summary_allsctld %>%
  select(-health_status) %>%
  summarise(across(everything(), sum)) %>%
  mutate(health_status = "Total") %>%
  select(everything())

# Combine
allsctld_summarytable <- bind_rows(summary_allsctld, allstatus_totals) %>%
    arrange(factor(health_status, levels = c('Resistant','Susceptible','Sus_recovery','Sus_mortality','Total')))
allsctld_summarytable

health_status,CNAT,DLAB,MCAV,MMEA,OANN,OFAV,PAST,PSTR,SSID,Total
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Resistant,1,7,13,0,5,8,16,8,5,63
Susceptible,0,0,0,0,3,3,0,1,18,25
Sus_recovery,0,1,1,0,0,0,4,0,2,8
Sus_mortality,0,0,17,13,0,0,0,22,3,55
Total,1,8,31,13,8,11,20,31,28,151


In [ ]:
# export tables 
# write.csv(allsctld_summarytable, 'colony_tables/sctld_colonysummary.csv',row.names=FALSE)

In [70]:
# excluding cnat, dlab, ssid - do my numbers match 
151-(1+8+28)
# yes i have 113 and i think the AS colony didn't get taken out of this dataset 

[1] 114